# Get Images for Transfer Learning

Images has to be labelled and bias should be minimised. Using these datasets but only the relevant classes are taken:


In [ ]:
import shutil
import pandas as pd

def copyImages(file_str, src, dest):
    file = pd.read_csv(file_str, delim_whitespace=True)
    
    for index, row in file.iterrows():
        if(row[1] == 1):
            shutil.copyfile((src + str(row[0]) + ".jpg"),(dest + str(row[0])) + ".jpg")
            
copyImages("aeroplane_trainval.txt", "JPEGImages/", "aeroplane/")

copyImages("bicycle_trainval.txt", "JPEGImages/", "bicycle/")

copyImages("bird_trainval.txt", "JPEGImages/", "bird/")

copyImages("boat_trainval.txt", "JPEGImages/", "boat/")

copyImages("bottle_trainval.txt", "JPEGImages/", "bottle/")

copyImages("cat_trainval.txt", "JPEGImages/", "cat/")

copyImages("chair_trainval.txt", "JPEGImages/", "chair/")

copyImages("diningtable_trainval.txt", "JPEGImages/", "diningtable/")

copyImages("person_trainval.txt", "JPEGImages/", "person/")

copyImages("pottedplant_trainval.txt", "JPEGImages/", "pottedplant/")

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np

import tensorflow as tf
keras = tf.keras
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
import pathlib

data_root = pathlib.Path("./data")

import random

all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)
image_count = len(all_image_paths)

label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
print(label_names)
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [299, 299])
    image /= 255.0  # normalize to [0,1] range
    image -= 0.5
    image *= 2.
    
    return image

def load_and_preprocess_image(path):
    image = tf.read_file(path)
    return preprocess_image(image)

path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

print('image shape: ', image_label_ds.output_shapes[0])
print('label shape: ', image_label_ds.output_shapes[1])
print('types: ', image_label_ds.output_types)
print(image_label_ds)


['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'cat', 'chair', 'diningtable', 'person', 'pottedplant']
image shape:  (299, 299, 3)
label shape:  ()
types:  (tf.float32, tf.int64)
<DatasetV1Adapter shapes: ((299, 299, 3), ()), types: (tf.float32, tf.int64)>


In [3]:
BATCH_SIZE = 32

# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)

steps_per_epoch = int(np.ceil(len(all_image_paths)/BATCH_SIZE))
print(steps_per_epoch)

330


In [4]:
# Create pre-trained model
inceptionv3 = keras.applications.inception_v3.InceptionV3(weights='imagenet', input_shape=(299, 299, 3), include_top=False)

# Add global spatial average pooling layer
x = inceptionv3.output
x = keras.layers.GlobalAveragePooling2D()(x)

# Add a fully-connected layer to the raw output of network
x = keras.layers.Dense(512, activation='relu')(x)

# Add a drop-out layer
x = keras.layers.Dropout(rate=0.5)(x)

# Add a logistic layer (softmax) - to predict 10 classes
predictions = keras.layers.Dense(10, activation='softmax')(x)

# Compose the model based on new top-layer
new_inceptionv3 = keras.models.Model(inputs=inceptionv3.input, outputs=predictions)

# First: Train the new top-layer only
# Hence, freeze all layers in pre-trained model
for layer in inceptionv3.layers:
    layer.trainable = False

# Compile model, ready to be trained on new data
new_inceptionv3.compile(optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9, decay=0.0001), loss='categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Training the Model

In [5]:
#new_inceptionv3 = keras.models.load_model('inceptionv3_10classes_1.h5')

# Train the model
new_inceptionv3.fit(ds, epochs=1, steps_per_epoch=steps_per_epoch)
new_inceptionv3.save('inceptionv3_10classes_1.h5')

# Top-layers are trained
# Now start fine-tuning the convolutional layers, freeze the bottom N layers and train the remaining top layers
# But first, lets visualise the layers available in Inception v3
for i, layer in enumerate(inceptionv3.layers):
    print(i, layer.name)
    
# Choose N layers to freeze and unfreeze the rest
N = 249
for layer in new_inceptionv3.layers[:N]:
    layer.trainable = False
for layer in new_inceptionv3.layers[N:]:
    layer.trainable = True
    
# Recompile model for modifications to take effect
# Use Stochastic Gradient Descent with a Low learning rate this time
new_inceptionv3.compile(optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.00001), loss='categorical_crossentropy')

# Train the model again, this time fine-tuning some inception blocks alongside new top layers
new_inceptionv3.fit(ds, epochs=1, steps_per_epoch=steps_per_epoch)
new_inceptionv3.save('inceptionv3_10classes_1_fine.h5')

Instructions for updating:
Use tf.cast instead.
330/330 [==============================] - 1544s 5s/step - loss: 135.1009
0 input_1
1 conv2d
2 batch_normalization_v1
3 activation
4 conv2d_1
5 batch_normalization_v1_1
6 activation_1
7 conv2d_2
8 batch_normalization_v1_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_v1_3
13 activation_3
14 conv2d_4
15 batch_normalization_v1_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_v1_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_v1_6
24 batch_normalization_v1_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_v1_5
33 batch_normalization_v1_7
34 batch_normalization_v1_10
35 batch_normalization_v1_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_v1_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_v1_13
47 batch_normalization_v

## One-off Predictions

In [8]:
model = keras.models.load_model('inceptionv3_10classes_1.h5')
image = load_and_preprocess_image('test-data/bird.jpeg')
image = tf.expand_dims(image, axis=0)
print(image)
results = model.predict(image, batch_size=None, steps=1)
print(label_names)
print(results)

Tensor("ExpandDims_4:0", shape=(1, 299, 299, 3), dtype=float32)
['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'cat', 'chair', 'diningtable', 'person', 'pottedplant']
[[0.09999993 0.09999993 0.10000122 0.09999993 0.09999983 0.09999979
  0.0999998  0.09999982 0.09999987 0.09999991]]
